In [1]:
!pip install xgboost

In [2]:
import csv
import pandas as pd 
import numpy as np
from math import log
from sklearn.model_selection import train_test_split

In [42]:
test_data = pd.read_csv("./VU_DM_data/test_set_VU_DM.csv")
train_data = pd.read_csv("./VU_DM_data/training_set_VU_DM.csv")

In [43]:
train_data = train_data.drop(['comp1_inv','comp1_rate_percent_diff','comp2_inv','comp2_rate_percent_diff','comp3_inv','comp3_rate_percent_diff','comp4_inv','comp4_rate_percent_diff','comp5_inv','comp5_rate_percent_diff','comp6_inv','comp6_rate_percent_diff','comp7_inv','comp7_rate_percent_diff','comp8_inv','comp8_rate_percent_diff','gross_bookings_usd'], axis=1)
test_data = test_data.drop(['comp1_inv','comp1_rate_percent_diff','comp2_inv','comp2_rate_percent_diff','comp3_inv','comp3_rate_percent_diff','comp4_inv','comp4_rate_percent_diff','comp5_inv','comp5_rate_percent_diff','comp6_inv','comp6_rate_percent_diff','comp7_inv','comp7_rate_percent_diff','comp8_inv','comp8_rate_percent_diff'], axis=1)      


In [30]:
length = len(train_data['srch_id'])
length

4959183

# Fill NULL - Train

In [ ]:
searchIDs = train_data['srch_id']
date = train_data['date_time'].copy()
booking = np.array(train_data['srch_booking_window'])
length = len(train_data['date_time'])


In [ ]:
date

In [ ]:
# date time -> week no.
for i in range(len(date)):
    time = date[i]
    time = (time.split())[0]
    month = int(time[5:7])
    day = int(time[8:10])
    dictMonth = {1:31,2:59,3:90,4:120,5:151,6:181,7:212,8:243,9:273,10:304,11:334,12:365}
    num_week = (dictMonth[month] + day+ int(booking[i]) )
    if num_week>=365:
        num_week = num_week%365
    num_week = num_week//7
    if num_week > 51:
        num_week = num_week%51
    date[i] = num_week


In [ ]:
date.to_csv("./week_train.csv", index=False)

In [44]:
date = pd.read_csv("./week_train.csv")
date

,date_time
0,17
1,17
2,17
3,17
4,17
...,...
4958342,33
4958343,33
4958344,33
4958345,33


#### 更新date为week 

In [45]:
train_data['date_time'] = date['date_time']

#### 以下列把NULL替换成该列中位数

In [46]:
toFillNull = ['orig_destination_distance', 'visitor_hist_starrating','visitor_hist_adr_usd','prop_log_historical_price','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_query_affinity_score']
for i in toFillNull:
    train_data[i].fillna(train_data[i].median(), inplace = True)
    

#### 8个comp rate相加 => comp_score

In [53]:
toFillNullComp = ['comp1_rate','comp2_rate','comp3_rate','comp4_rate','comp5_rate','comp6_rate','comp7_rate','comp8_rate']
for i in toFillNullComp:
    train_data[i].fillna(0, inplace = True)
    

In [61]:
train_data["comp_score"] = train_data[toFillNullComp].sum(axis=1)

#### 删8个comp rate

In [68]:
train_data = train_data.drop(toFillNullComp, axis=1)
train_data

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,booking_bool,comp_score
0,1,17,12,187,3.45,152.24,219,893,3,3.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
1,1,17,12,187,3.45,152.24,219,10404,4,4.0,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
2,1,17,12,187,3.45,152.24,219,21315,3,4.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
3,1,17,12,187,3.45,152.24,219,27348,2,4.0,...,4,0,1,1,-20.4513,386.60,1,0,0,-3.0
4,1,17,12,187,3.45,152.24,219,29604,4,3.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,33,5,219,3.45,152.24,219,77700,3,4.0,...,3,0,1,0,-20.4513,550.92,0,0,0,0.0
4958343,332785,33,5,219,3.45,152.24,219,88083,3,4.0,...,3,0,1,0,-20.4513,553.14,0,0,0,0.0
4958344,332785,33,5,219,3.45,152.24,219,94508,3,3.5,...,3,0,1,0,-20.4513,544.43,0,0,0,0.0
4958345,332785,33,5,219,3.45,152.24,219,128360,3,5.0,...,3,0,1,0,-20.4513,550.38,0,1,1,0.0


In [69]:
train_data.to_csv("./clean_train.csv", index=False)

# Fill NULL - Test 同上

In [18]:
test_date = test_data['date_time'].copy()
test_booking = np.array(test_data['srch_booking_window'])
test_length = len(test_data['srch_id'])
test_date

0          2013-02-02 15:27:40
1          2013-02-02 15:27:40
2          2013-02-02 15:27:40
3          2013-02-02 15:27:40
4          2013-02-02 15:27:40
                  ...         
4959178    2013-05-21 11:06:37
4959179    2013-05-21 11:06:37
4959180    2013-05-21 11:06:37
4959181    2013-05-21 11:06:37
4959182    2013-05-21 11:06:37
Name: date_time, Length: 4959183, dtype: object

In [19]:
# date time -> week no.
for i in range(len(test_date)):
    test_time = test_date[i]
    test_time = (test_time.split())[0]
    test_month = int(test_time[5:7])
    test_day = int(test_time[8:10])
    dictMonth = {1:31,2:59,3:90,4:120,5:151,6:181,7:212,8:243,9:273,10:304,11:334,12:365}
    test_num_week = (dictMonth[test_month] + test_day+ int(test_booking[i]) )
    if test_num_week>=365:
        test_num_week = test_num_week%365
    test_num_week = test_num_week//7
    if test_num_week > 51:
        test_num_week = test_num_week%51
    test_date[i] = test_num_week
    

In [21]:
test_date.to_csv("./week_test.csv", index=False)

In [70]:
test_date = pd.read_csv("./week_test.csv")

In [71]:
toFillNull = ['orig_destination_distance', 'visitor_hist_starrating','visitor_hist_adr_usd','prop_log_historical_price','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_query_affinity_score']
for i in toFillNull:
    test_data[i].fillna(test_data[i].median(), inplace = True)

In [72]:
test_data['date_time'] = test_date

In [74]:
toFillNullComp = ['comp1_rate','comp2_rate','comp3_rate','comp4_rate','comp5_rate','comp6_rate','comp7_rate','comp8_rate']
for i in toFillNullComp:
    test_data[i].fillna(0, inplace = True)
test_data["comp_score"] = test_data[toFillNullComp].sum(axis=1)



In [77]:
test_data = test_data.drop(toFillNullComp, axis=1)

In [79]:
test_data.to_csv("./clean_test.csv", index=False)

# Get clean data

In [80]:
test_data = pd.read_csv("./clean_test.csv")
train_data = pd.read_csv("./clean_train.csv")

In [81]:
train_data

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,click_bool,booking_bool,comp_score
0,1,17,12,187,3.45,152.24,219,893,3,3.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
1,1,17,12,187,3.45,152.24,219,10404,4,4.0,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
2,1,17,12,187,3.45,152.24,219,21315,3,4.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
3,1,17,12,187,3.45,152.24,219,27348,2,4.0,...,4,0,1,1,-20.4513,386.60,1,0,0,-3.0
4,1,17,12,187,3.45,152.24,219,29604,4,3.5,...,4,0,1,1,-20.4513,386.60,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4958342,332785,33,5,219,3.45,152.24,219,77700,3,4.0,...,3,0,1,0,-20.4513,550.92,0,0,0,0.0
4958343,332785,33,5,219,3.45,152.24,219,88083,3,4.0,...,3,0,1,0,-20.4513,553.14,0,0,0,0.0
4958344,332785,33,5,219,3.45,152.24,219,94508,3,3.5,...,3,0,1,0,-20.4513,544.43,0,0,0,0.0
4958345,332785,33,5,219,3.45,152.24,219,128360,3,5.0,...,3,0,1,0,-20.4513,550.38,0,1,1,0.0


# xgboost 开始训练，代码还没怎么改，基本一致

In [ ]:
new_train_data.columns

In [ ]:
new_test_data.columns

In [ ]:
train_data.groupby('srch_room_count')['srch_id', 'prop_id'].nunique()

In [ ]:
new_train_data.describe(exclude=[np.number])

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df2 = new_train_data.copy()
for column in df2.select_dtypes(exclude=[np.number]).columns:
    df2[column] = le.fit_transform(df2[column].astype(str))
# df2.dropna(inplace=True)
train_data = df2.copy()


In [ ]:
df3 = new_test_data.copy()
for column in df3.select_dtypes(exclude=[np.number]).columns:
    df3[column] = le.fit_transform(df3[column].astype(str))
# df3.dropna(inplace=True)
test_data = df3.copy()


In [ ]:
from sklearn.utils import shuffle

numericData = train_data.copy()
numericData = shuffle(numericData)
numericData.reset_index(inplace=True, drop=True)
x = numericData.drop(['booking_bool','click_bool','position'], axis=1)
y = numericData['booking_bool']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42, shuffle=True)

In [ ]:
weighting = numericData.groupby(['booking_bool']).size()
weighting

In [ ]:
scale_pos_weight = weighting[0]/weighting[1]
scale_pos_weight

In [ ]:
import xgboost as xgb
from xgboost import DMatrix, train


training_group = x.groupby(["srch_id"]).size().values
xgb_train = DMatrix(x, label=y)
xgb_train.set_group(training_group)

evaluation_set = [(xgb_train, 'train')]

In [ ]:
%time ranking_model = train({}, xgb_train, evals=evaluation_set, num_boost_round=1000, early_stopping_rounds=10)

In [ ]:
ranking_model = train({}, xgb_train, evals=evaluation_set, num_boost_round=100, xgb_model=ranking_model)

In [ ]:
import matplotlib.pyplot as plt

xgc = ranking_model
# saving the feature names to the model
xgc.feature_names = x_train.columns.to_list()
# Create the feature importances plot
fig, ax = plt.subplots(1, 3, figsize=(15,5))
# plot importances with feature weight
xgb.plot_importance(
    booster=xgc, 
    importance_type='weight',
    title='Feature Weight',
    show_values=False,
    height=0.5,
    ax=ax[0],
)
# plot importances with split mean gain
xgb.plot_importance(
    booster=xgc,
    importance_type='gain',
    title='Split Mean Gain',
    show_values=False,
    height=0.5,
    ax=ax[1]
)
# plot importances with sample coverage
xgb.plot_importance(
    booster=xgc,
    importance_type='cover',
    title='Sample Coverage',
    show_values=False,
    height=0.5,
    ax=ax[2]
)
plt.tight_layout()
plt.show()

In [ ]:
newX = x.drop(['srch_saturday_night_bool'], axis=1)


In [ ]:
import xgboost as xgb
from xgboost import DMatrix, train

training_group = newX.groupby(["srch_id"]).size().values
xgb_train = DMatrix(newX, label=y)
xgb_train.set_group(training_group)

evaluation_set = [(xgb_train, 'train')]

In [ ]:
%time ranking_model_reduced = train({}, xgb_train, evals=evaluation_set, num_boost_round=100)

In [ ]:
%time ranking_model_reduced = train({}, xgb_train, evals=evaluation_set, num_boost_round=10, xgb_model=ranking_model_reduced)


In [ ]:
mat = DMatrix(test_data)
predictions = ranking_model.predict(mat)
testingNumericData = test_data.copy()
testingNumericData['predictions'] = predictions
results = testingNumericData[["srch_id", "prop_id", "predictions"]]

In [ ]:
results_to_csv = results.sort_values(['srch_id'], ascending=True).groupby(['srch_id'], sort=False).apply(lambda x: x.sort_values(['predictions'], ascending=False)).reset_index(drop=True)


In [ ]:
submissions = results_to_csv[["srch_id", "prop_id"]]
submissions.to_csv("./submission2.csv", index=False)